In [2]:
!pip install -r requirements.txt

     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/65.6 kB ? eta -:--:--
     ----------------- -------------------- 30.7/65.6 kB 187.9 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/65.6 kB 164.3 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/65.6 kB 164.3 kB/s eta 0:00:01
     ----------------------------------- -- 61.4/65.6 kB 193.2 kB/s eta 0:00:01
     -------------------------------------- 65.6/65.6 kB 186.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- --------

In [4]:
from openai import OpenAI
from dotenv import load_dotenv , find_dotenv

_:bool = load_dotenv(find_dotenv())

client:OpenAI = OpenAI()


In [2]:
from openai.types.chat.chat_completion import ChatCompletion
def chat_completion(prompt:str) -> str:
    response : ChatCompletion = client.chat.completions.create(
        model= "gpt-3.5-turbo-1106",
        messages=[{
            "role":"user",
            "content": prompt
        }]
    
    )
    return response.choices[0].message.content
result = chat_completion("what is 2 + 10")
print(result)

2 + 10 equals 12.


# Assistant API

In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [5]:
assistant = client.beta.assistants.create(
    name="GYM Instructor",
    model="gpt-3.5-turbo-1106",
    instructions="You are a GYM instructor and you answer every question about fitness, diet and exercise."
)
show_json(assistant)

{'id': 'asst_jPEwjqDydtks4yZOyweXv3rv',
 'created_at': 1704556103,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a GYM instructor and you answer every question about fitness, diet and exercise.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'GYM Instructor',
 'object': 'assistant',
 'tools': []}

In [6]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG',
 'created_at': 1704556109,
 'metadata': {},
 'object': 'thread'}

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="my weigth is 95kg and my height is 5.7feet and i am 17 years old suggest me a diet to reduce my weight to 65kg?"  
)
show_json(message)

{'id': 'msg_jOchdVBvA33mvkqj7kTzoT08',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'my weigth is 95kg and my height is 5.7feet and i am 17 years old suggest me a diet to reduce my weight to 65kg?'},
   'type': 'text'}],
 'created_at': 1704556112,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG'}

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what is my weight in lbs?"  
)

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    
)
show_json(run)

{'id': 'run_jsxFZT8ChiCZuBno9UkMB6gR',
 'assistant_id': 'asst_jPEwjqDydtks4yZOyweXv3rv',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1704556117,
 'expires_at': 1704556717,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a GYM instructor and you answer every question about fitness, diet and exercise.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG',
 'tools': []}

In [10]:
import time
def wait_on_run(run,thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        time.sleep(0.5)
    return run


In [11]:
lol = wait_on_run(run,thread)
show_json(lol)

{'id': 'run_jsxFZT8ChiCZuBno9UkMB6gR',
 'assistant_id': 'asst_jPEwjqDydtks4yZOyweXv3rv',
 'cancelled_at': None,
 'completed_at': 1704556122,
 'created_at': 1704556117,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a GYM instructor and you answer every question about fitness, diet and exercise.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1704556117,
 'status': 'completed',
 'thread_id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG',
 'tools': []}

In [12]:
answer = client.beta.threads.messages.list(thread_id=thread.id)
show_json(answer)
for m in reversed(answer.data):
    print(m.role + ": " + m.content[0].text.value)

{'data': [{'id': 'msg_8F00GdKJKd8dX6D9TdUmYVDc',
   'assistant_id': 'asst_jPEwjqDydtks4yZOyweXv3rv',
   'content': [{'text': {'annotations': [],
      'value': 'To convert your weight from kilograms to pounds, you can multiply your weight in kilograms by 2.20462. \n\nSo, 95 kg * 2.20462 = 209.43 lbs. \n\nYour weight is approximately 209 lbs.'},
     'type': 'text'}],
   'created_at': 1704556120,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_jsxFZT8ChiCZuBno9UkMB6gR',
   'thread_id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG'},
  {'id': 'msg_l7uWot9fAW3EUxa5CBosfbYj',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'what is my weight in lbs?'},
     'type': 'text'}],
   'created_at': 1704556114,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_E4OjGHyr03VKVG0z3IYgvNTG'},
  {'id': 'msg_jOchdVBvA33mvkqj7kTzoT08',
   '

user: my weigth is 95kg and my height is 5.7feet and i am 17 years old suggest me a diet to reduce my weight to 65kg?
user: what is my weight in lbs?
assistant: To convert your weight from kilograms to pounds, you can multiply your weight in kilograms by 2.20462. 

So, 95 kg * 2.20462 = 209.43 lbs. 

Your weight is approximately 209 lbs.
